# Introducing the fumbbl_replays Python package

The `fumbbl_replays` Python package is about analyzing FUMBBL game logs for the board game Blood Bowl.
On the FUMBBL website, a lot of high quality replay data is available as well as an API to conveniently fetch the data.
In addition, the API provides up to date roster information.
To use do useful analyses (aka nufflytics) in Python with this data, we need a utility package / library.
In R, a similar package exists to work with BB2 replays [https://github.com/nufflytics/nufflytics].
To my knowledge, no work has been done yet on BB3 replay files.

We also need a standard way to describe Blood Bowl games in a compact way, that is both human and machine readable.
In chess, there is the **Portable Game Notation (PGN)**. PGN has become the de facto standard of describing Chess games.
For Blood Bowl, already in 2002 some work has been done towards this end. David Morgan-Mar developed a notation for the purpose of sharing game logs over the internet. [https://www.dangermouse.net/games/bloodbowl/rules.html]

If we could converge on a standard Fantasy Football Game Notation, it would serve many purposes:
* It would allow us to interchange data between software
* it would help to train AI engines.

So the package is being developed with this end goal in mind. As Blood Bowl is a much complexer game than Chess, and I am an amateur programmer, we need some intermediate goals that bring us closer to the end goal. Thus, I started with extracting FUMBBL board positions, plotting board positions using a short hand notation for board pieces (players) and codifying player moves.

In [ ]:
%pip install -e . --quiet

# Plotting Blood Bowl board positions

If we want to describe (codify) a Blood Bowl board state, we need to describe the pieces (what type of player is it, what extra skills does it have), the location of the pieces as well as the "state" of the pieces. Player state in Blood Bowl can be either standing, prone, or stunned, and can be in various special states such as "Bone head", "Rooted", "Hypnotized" etc. 
(A full game state also contains additional information on rerolls, players on the bench etc. This is not yet implemented)

Let's start with the location of the pieces. A grid reference system is needed.
The game board of Blood Bowl has dimensions 15 x 26.
It has cognitive benefit to use numbers for one dimension, and letters for the other dimensions. Fancy word: alphanumeric.
Chess over the centuries has had various notations, and this notation is the one that became universally accepted.
[https://en.wikipedia.org/wiki/Algebraic_notation_(chess)]
The only choice left for us is then, which axis should have letters, and which axis should have the numbers.

A strong argument was made on the BotBowl discord that distance to the end zone is very important in BB.
By using numbers for the long axis, we can easily deduce that a Gutter Runner at position c15 is in scoring position: It needs 11 movement to score a touchdown at c26.
This notation is also used by Cow Daddy Gaming in his "What´s the play" puzzles.

I wrote a function `show_boardpos()` that displays the name of all the board positions.


In [ ]:
import fumbbl_replays as fb

fb.show_boardpos(rotation = 'H')

Next is that we need a way to describe the playing pieces, and visualize them. In chess it is easy, there are only six different ones.
In Blood Bowl, there are roughly 200 different playing pieces (30 teams, times 5 positionals, plus 50+ star players).
Here the concept of a roster can help us out. 
I wrote a function `fetch_roster()` that fetches rosters from FUMBBL and displays the positions.
It also contains links to icons that can represent the piece on the board.

Take for example the High Elf roster.

In [ ]:
roster = fb.fetch_roster("High Elf")
roster

It has four different pieces or "positionals". It turns out that FUMBBL has already solved our problem of denoting them, introducing a shorthand text reference. 
So if we want to describe some action involving a High Elf Catcher, and there are four of them on the board, we could denote them by C1, C2, C3 and C4.
This is compact, and has meaning within the context of the High Elf roster.

If we combine the descriptions of the pieces, and their location, we have enough to describe for example an initial setup formation before kick-off.

In [ ]:
roster = fb.fetch_roster("High Elf")

my_setup = ['setup', ['L1: g13', 'L2: h13', 'L3: i13', 'Z1: c11', 'Z2: m11', 'T1: h6', 'L4: e11', 
                      'L5: k11', 'C1: l10', 'C2: d10', 'L6: h11']]

I wrote a function `create_position()` that combines the roster and the setup annotation to create an object that contains all the information to make a nice plot of the board state. The function `print_position()` prints a nicely formatted summary of the position.
As default, a position is created for the home team, denoted as "teamHome".

In [ ]:
positions = fb.create_position(roster, my_setup)
fb.print_position(positions)

Let's suppose that the High Elf team is playing against a Gnome team. Let's also fetch a Gnome roster and create a board position on the other half of the pitch.
As we already have a home team, we refer to this team as "teamAway".

In [ ]:
roster = fb.fetch_roster("Gnome")
roster

In [ ]:
my_setup = ['setup', ['T2: j14', 'T1: f14', 'F1: h20', 'I1: b14', 'I2: n14', 'L3: e14', 'L6: k14', 
                      'B2: m15', 'B1: c15', 'L4: g15', 'F2: i16']]

positions2 = fb.create_position(roster, my_setup, 'teamAway')

fb.print_position(positions2)

In [ ]:
positions2

As a final step before plotting, we add both positions together.
As both are `pandas` DataFrames, we use the `concat()` function from `pandas`  to combine ("concatenate") them.

In [ ]:
import pandas as pd

positions = pd.concat([positions, positions2])

The function `create_plot()` plots the board position.
By default, it plots a horizontal pitch, with the team denoted as "teamHome" in red, and the other team in blue.

In [ ]:
fb.create_plot(positions)

The `create_plot()` function allows us the swap the color of the teams, to change the pitch orientation to vertical, and to add a layer of semi-transparant tacklezones.

In [ ]:
fb.create_plot(positions, red_team = "teamAway", orientation = 'V', tackle_zones = True)

The library also support moving single pieces (players). It currently only works for pieces that already exist in a board position.
In the plot above, suppose we want to move the Woodland Fox F1 to board position `o26`:

In [ ]:
positions = fb.move_piece(positions, "teamAway", "F1", "o26")

fb.create_plot(positions, red_team = "teamAway")

Each player also has an associated `PlayerState`. This can either be `Standing`, `HasBall`, `Prone` or `Stunned`.
The function `set_piece_state()` allows to set this for individual players:

In [ ]:
positions = fb.set_piece_state(positions, "teamAway", "F1", "HasBall")
positions = fb.set_piece_state(positions, "teamHome", "T1", "Prone")
positions = fb.set_piece_state(positions, "teamAway", "B1", "Stunned")

fb.create_plot(positions, red_team = "teamAway")

Finally, we can get a compact description of this board position using `get_position()`:

In [ ]:
fb.get_position(positions, home_away = 'teamAway')

In [ ]:
# PM add set_color_boardpos()


# Plotting board positions from FUMBBL replays

Up until now, we created board positions from scratch, using rosters from FUMBBL and a simple way to describe a board position.


The package also allows us to plot board positions extracted from FUMBBL replay files.
At this moment, only the board position right before kick-off can be plotted.
Suppose we want to plot this position for [match 4551601](https://www.fumbbl.com/p/match?id=4551601).

We first need to fetch the replay data. The `fetch_data()` function takes the `match_id` as argument and returns five objects:
the `match_id`, `replay_id`, a `positions` object, which team is the `receiving_team` (i.e. playing offense), and a `metadata` list (coach names, race names, and match touchdown result).

In [ ]:
match_id, replay_id, positions, receiving_team, metadata = fb.fetch_data(match_id = 4550284)

To plot the board state right before kick-off, we can use the `create_plot()` function in the same way as above.
We plot the receiving team in red so we can see which team is playing offense and which team is playing defense.

In [ ]:
fb.create_plot(positions, red_team = receiving_team)

In [ ]:
(positions.filter(['race', 'home_away', 'short_name', 'positionName', 'playerName',  'skillArrayRoster', 'learned_skills', 'cost', 'recoveringInjury'])
                    )

Adjusting this board position by moving players one-by-one works also in the same way as above.

In [ ]:
positions = fb.move_piece(positions, "teamAway", "Z1", "b26")
positions = fb.move_piece(positions, "teamAway", "Z2", "o26")

fb.create_plot(positions, red_team = receiving_team)

Suppose we think that the Gnome defensive setup is awesome, and we wish to share this setup with other coaches.
Here the compact way to describe a setup using player abbreviations and the alphanumeric grid system comes in handy.
To get the position in this notation I wrote the function `get_position()`.


In [ ]:
fb.get_position(positions, home_away = 'teamHome')


However, suppose we think this setup is nice, but it would be even better if the illusionist in row `g` would be a  Woodland Fox.
We can take the setup (copy-paste), change the setup slightly, and create a new position, with `F1: g10`.
As we now only have a single team, we can rotate the pitch and crop to show only the upper part of it.

In [ ]:
roster = fb.fetch_roster("Gnome")

my_setup = ['setup', ['T2: g13', 'T1: i13', 'L2: h13', 'B1: f10', \
                      'L1: e11', 'B2: j10', 'L4: k11', 'L5: h11', \
                        'L7: i10', 'F1: g10', 'L3: h10']]

positions = fb.create_position(roster, my_setup)

fb.create_plot(positions, orientation= "V", crop = "upper")

# Working with raw replays directly

It is also possible to work with the raw FUMBBL replay files directly.
I made a start with describing the replay file format in `doc/fumbbl_replay_file_format.md`.
We can use `fetch_replay()` to retrieve a replay in JSON format.

In [ ]:
import fumbbl_replays as fb

my_replay = fb.fetch_replay(match_id = 4447439)

JSON consists of key-value pairs.
We can for example query the value of the key `gameStatus`:

In [ ]:
my_replay['gameStatus']

Or query the `rosterName`:

In [ ]:
my_replay['game']['teamHome']['roster']['rosterName']

The replay contains both a game log, as well as full roster information on both teams.
We can extract the roster information from the replay using the function `extract_rosters_from_replay()`.


In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df_positions = fb.extract_rosters_from_replay(my_replay)
(df_positions
 .query("home_away == 'teamAway'")
 .filter(['short_name', 'positionName', 'skillArrayRoster', 'learned_skills', 'skill_colors'])
)

I wrote a replay parser that parses the gameLog section of a replay and transforms this into a `pandas` DataFrame object, i.e. a flat 2D table with rows and columns.

In [ ]:
df = fb.parse_replay(my_replay)
(df[0:4]
 .filter(['commandNr', 'turnNr', 'turnMode', 'Half', 'modelChangeId', 'modelChangeValue'])
)

We can use the `pandas` `query()` function to select rows based on conditions.
This query selects all "fieldModelSetPlayerCoordinate" commands during setup before turn 1.

In [ ]:
positions = (df.query('turnNr == 0 & turnMode == "setup" & Half == 1 & \
                     modelChangeId == "fieldModelSetPlayerCoordinate"')
                     .groupby('modelChangeKey')
                     .tail(1))

(positions[0:4]
 .filter(['commandNr', 'turnNr', 'turnMode', 'Half', 'modelChangeId', 'modelChangeValue'])
)

As I was interested in **defensive** setup formations, I wrote a function `determine_receiving_team_at_start()` that does exactly what you'd expect given its name :)

In [ ]:
fb.determine_receiving_team_at_start(df)

# Towards FFGN

Finally, there is a function `fumbbl2ffgn()` that is very much a work in progress.
The idea is to take a FUMBBL game log, and systematically strip away all information that is redundant regarding the actual logging of what happened during the game. A minimal game description would consist of all actions taken, all decisions that were made (i.e. to use the dodge skill) and all dice results.
After we have such a description, we can transform it to a compact annotation that is readable both by humans and machines, and is still a complete description of the game, in that the full game can be reproduced.
The compact annotation would then be candidate to become the official "Fantasy Football Game Notation", or FFGN for short.

In [ ]:
my_game_log = fb.fumbbl2ffgn(match_id = 4447439)
len(my_game_log)

This is where it currently stands. A single gamelog is now roughly 1000 lines of text.
The table below describes the first turn of a Wood Elf team against Necromantic.

In [ ]:
pd.set_option('display.max_colwidth', None)

# Turn 1 for the offensive
(my_game_log
 .query("Half == 1 & turnNr == 1 & commandNr > 88 & commandNr < 211")
 .filter(['modelChangeKey', 'modelChangeValue'])
)

*Thrower 1 moves. Catcher 1 moves. Lineman 1 moves. Lineman 4 blocks, chooses pow/push, pows Zombie lineman L5 into square g12, follows up, does not break armor.*
*Treeman does not take root, does a 3D block on Zombie lineman 2, chooses pow into square i12, follows up to square h13, does not break armor.*
*Lineman 5 blocks zombie lineman L4, pows into h12, does not follow up, does not break armor. Then linemen L3, L2, catcher C2 and wardancer W1 all do a move action.*
*Finally Wardancer W2 moves, picks up the ball and moves a bit more. End turn.*

This blog post describes the basic functionality. I have a second blog post that contains three applications that use the package to accomplish some nufflytics goal.
